CIFAR-10 is loaded using the keras datasets interface. The per-pixel mean is subtracted as preprocessing. The code in this notebook is based on the keras example found here: https://github.com/keras-team/keras/blob/master/examples/cifar10_resnet.py

In [1]:
import os
import numpy as np
import keras
from keras.datasets import cifar10

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

Using TensorFlow backend.


The model is created the same way as cifar10_resnet.py does. Checkpoints are saved in the "saved_models" directory. The model itself is a 29 layer deep version of resnet v2. More details can be found in cifar10_resnet.py.

In [2]:
from keras.layers import *
from model import LogGaussMF
from resnet_backend import *


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', 1. * lr)
    return 1. * lr


# Prepare model model saving directory.
def get_callbacks(model_type):
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    model_name = "%s_model.{epoch:03d}.h5" % model_type
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    filepath = os.path.join(save_dir, model_name)

    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(
        filepath=filepath,
        monitor='val_acc',
        verbose=1,
        save_best_only=True)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(
        factor=np.sqrt(0.1),
        cooldown=0,
        patience=5,
        min_lr=0.5e-6)

    return [checkpoint, lr_reducer, lr_scheduler]


use_anfis = True

n = 6
depth = n * 9 + 2

# Model name, depth and version
model_type = 'Fuzzy-ResNet%dv%d' % (depth, 2)

inputs, features = resnet_backend_v2(
    input_shape=input_shape, 
    depth=depth)

memberships = LogGaussMF(10)(features)
rules = Lambda(lambda x: K.sum(x, axis=-1))(memberships)

if use_anfis:
    linear = Dense(10)(features)
    logits = Add()([rules, linear])
else:
    logits = rules

outputs = Activation("softmax")(logits)
model = Model(inputs=inputs, outputs=outputs)

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=lr_schedule(0)),
    metrics=['accuracy'])

callbacks = get_callbacks(model_type)

W0508 13:17:21.270412 140181672830784 deprecation.py:506] From /home/ryan-desktop/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/training/moving_averages.py:210: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Learning rate:  0.001


Data augmentation is used to improve performance. The model took approximately 3-3.5 hours to train and reached an accuracy of 92.25%

In [3]:
datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(
    datagen.flow(x_train, y_train, batch_size=32),
    steps_per_epoch=60000 // 32,
    validation_data=(x_test, y_test),
    epochs=200, 
    verbose=1, 
    workers=4,
    callbacks=callbacks)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/200
Learning rate:  0.001
1875/1875 [==============================] - 124s 66ms/step - loss: 1.9876 - acc: 0.5071 - val_loss: 1.8783 - val_acc: 0.5161

Epoch 00001: val_acc improved from -inf to 0.51610, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/Fuzzy-ResNet56v2_model.001.h5
Epoch 2/200
Learning rate:  0.001
1875/1875 [==============================] - 113s 60ms/step - loss: 1.3460 - acc: 0.6592 - val_loss: 1.7342 - val_acc: 0.5742

Epoch 00002: val_acc improved from 0.51610 to 0.57420, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/Fuzzy-ResNet56v2_model.002.h5
Epoch 3/200
Learning rate:  0.001
1875/1875 [==============================] - 112s 60ms/step - loss: 1.1510 - acc: 0.7166 - val_loss: 1.3629 - val_acc: 0.6507

Epoch 00003: val_acc improved from 0.57420 to 0.65070, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/Fuzzy-ResNet56v2_model.003.h5
Epoch 4/200
Learning r

1875/1875 [==============================] - 106s 57ms/step - loss: 0.4590 - acc: 0.9081 - val_loss: 0.7075 - val_acc: 0.8393

Epoch 00070: val_acc did not improve from 0.87110
Epoch 71/200
Learning rate:  0.001
1875/1875 [==============================] - 106s 57ms/step - loss: 0.4518 - acc: 0.9112 - val_loss: 0.8095 - val_acc: 0.8171

Epoch 00071: val_acc did not improve from 0.87110
Epoch 72/200
Learning rate:  0.001
1875/1875 [==============================] - 106s 56ms/step - loss: 0.4503 - acc: 0.9113 - val_loss: 0.6926 - val_acc: 0.8421

Epoch 00072: val_acc did not improve from 0.87110
Epoch 73/200
Learning rate:  0.001
1875/1875 [==============================] - 106s 57ms/step - loss: 0.4582 - acc: 0.9083 - val_loss: 0.6161 - val_acc: 0.8626

Epoch 00073: val_acc did not improve from 0.87110
Epoch 74/200
Learning rate:  0.001
1875/1875 [==============================] - 106s 57ms/step - loss: 0.4493 - acc: 0.9118 - val_loss: 0.6587 - val_acc: 0.8508

Epoch 00074: val_acc did 

1875/1875 [==============================] - 106s 57ms/step - loss: 0.2109 - acc: 0.9752 - val_loss: 0.4015 - val_acc: 0.9225

Epoch 00104: val_acc did not improve from 0.92580
Epoch 105/200
Learning rate:  0.0001
1875/1875 [==============================] - 106s 57ms/step - loss: 0.2077 - acc: 0.9755 - val_loss: 0.4064 - val_acc: 0.9196

Epoch 00105: val_acc did not improve from 0.92580
Epoch 106/200
Learning rate:  0.0001
1875/1875 [==============================] - 106s 57ms/step - loss: 0.2066 - acc: 0.9751 - val_loss: 0.4020 - val_acc: 0.9194

Epoch 00106: val_acc did not improve from 0.92580
Epoch 107/200
Learning rate:  0.0001
1875/1875 [==============================] - 106s 56ms/step - loss: 0.2023 - acc: 0.9761 - val_loss: 0.3967 - val_acc: 0.9206

Epoch 00107: val_acc did not improve from 0.92580
Epoch 108/200
Learning rate:  0.0001
1875/1875 [==============================] - 106s 56ms/step - loss: 0.2009 - acc: 0.9765 - val_loss: 0.3937 - val_acc: 0.9198

Epoch 00108: val_

1875/1875 [==============================] - 106s 57ms/step - loss: 0.1533 - acc: 0.9879 - val_loss: 0.3781 - val_acc: 0.9283

Epoch 00141: val_acc did not improve from 0.92930
Epoch 142/200
Learning rate:  1e-05
1875/1875 [==============================] - 106s 56ms/step - loss: 0.1531 - acc: 0.9878 - val_loss: 0.3805 - val_acc: 0.9272

Epoch 00142: val_acc did not improve from 0.92930
Epoch 143/200
Learning rate:  1e-05
1875/1875 [==============================] - 106s 56ms/step - loss: 0.1532 - acc: 0.9877 - val_loss: 0.3789 - val_acc: 0.9278

Epoch 00143: val_acc did not improve from 0.92930
Epoch 144/200
Learning rate:  1e-05
1875/1875 [==============================] - 106s 56ms/step - loss: 0.1532 - acc: 0.9878 - val_loss: 0.3813 - val_acc: 0.9266

Epoch 00144: val_acc did not improve from 0.92930
Epoch 145/200
Learning rate:  1e-05
1875/1875 [==============================] - 106s 56ms/step - loss: 0.1531 - acc: 0.9874 - val_loss: 0.3780 - val_acc: 0.9281

Epoch 00145: val_acc 


Epoch 00179: val_acc did not improve from 0.92930
Epoch 180/200
Learning rate:  1e-06
1875/1875 [==============================] - 106s 57ms/step - loss: 0.1451 - acc: 0.9895 - val_loss: 0.3796 - val_acc: 0.9277

Epoch 00180: val_acc did not improve from 0.92930
Epoch 181/200
Learning rate:  1e-06
1875/1875 [==============================] - 106s 57ms/step - loss: 0.1453 - acc: 0.9892 - val_loss: 0.3790 - val_acc: 0.9283

Epoch 00181: val_acc did not improve from 0.92930
Epoch 182/200
Learning rate:  5e-07
1875/1875 [==============================] - 106s 57ms/step - loss: 0.1457 - acc: 0.9892 - val_loss: 0.3785 - val_acc: 0.9279

Epoch 00182: val_acc did not improve from 0.92930
Epoch 183/200
Learning rate:  5e-07
1875/1875 [==============================] - 105s 56ms/step - loss: 0.1466 - acc: 0.9890 - val_loss: 0.3779 - val_acc: 0.9284

Epoch 00183: val_acc did not improve from 0.92930
Epoch 184/200
Learning rate:  5e-07
1875/1875 [==============================] - 106s 56ms/step -

The modified version of resnet, replacing the classifier with the fuzzy classifier. It is trained with the same parameters as the normal version of resnet. It was able to achieve 92.56% accuracy, slightly better than the traditional version, but this may be due to random chance.

In [3]:
n = 6
depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, 2)

inputs, features = resnet_backend_v2(
    input_shape=input_shape, 
    depth=depth)
linear = Dense(10)(features)
outputs = Activation("softmax")(linear)
model = Model(inputs=inputs, outputs=outputs)

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=0.01),
    #optimizer=Adam(lr=lr_schedule(0)),
    metrics=['accuracy'])

callbacks = get_callbacks(model_type)[0:1]

In [4]:
datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(
    datagen.flow(x_train, y_train, batch_size=128),
    steps_per_epoch=60000 // 128,
    validation_data=(x_test, y_test),
    epochs=200, 
    verbose=1, 
    workers=4,
    callbacks=callbacks)

Epoch 1/200
Learning rate:  0.001
468/468 [==============================] - 67s 143ms/step - loss: 2.2154 - acc: 0.5002 - val_loss: 2.0443 - val_acc: 0.5297

Epoch 00001: val_acc improved from -inf to 0.52970, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/ResNet56v2_model.001.h5
Epoch 2/200
Learning rate:  0.001
468/468 [==============================] - 52s 110ms/step - loss: 1.5521 - acc: 0.6482 - val_loss: 1.6170 - val_acc: 0.6111

Epoch 00002: val_acc improved from 0.52970 to 0.61110, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/ResNet56v2_model.002.h5
Epoch 3/200
Learning rate:  0.001
468/468 [==============================] - 51s 109ms/step - loss: 1.2782 - acc: 0.7198 - val_loss: 1.6127 - val_acc: 0.6189

Epoch 00003: val_acc improved from 0.61110 to 0.61890, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/ResNet56v2_model.003.h5
Epoch 4/200
Learning rate:  0.001
468/468 [===

468/468 [==============================] - 53s 114ms/step - loss: 0.4877 - acc: 0.9140 - val_loss: 0.7281 - val_acc: 0.8483

Epoch 00032: val_acc did not improve from 0.86620
Epoch 33/200
Learning rate:  0.001
468/468 [==============================] - 52s 112ms/step - loss: 0.4841 - acc: 0.9164 - val_loss: 0.6701 - val_acc: 0.8628

Epoch 00033: val_acc did not improve from 0.86620
Epoch 34/200
Learning rate:  0.001
468/468 [==============================] - 52s 111ms/step - loss: 0.4755 - acc: 0.9174 - val_loss: 0.7138 - val_acc: 0.8514

Epoch 00034: val_acc did not improve from 0.86620
Epoch 35/200
Learning rate:  0.001
468/468 [==============================] - 51s 110ms/step - loss: 0.4759 - acc: 0.9174 - val_loss: 0.6362 - val_acc: 0.8673

Epoch 00035: val_acc improved from 0.86620 to 0.86730, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/ResNet56v2_model.035.h5
Epoch 36/200
Learning rate:  0.001
468/468 [==============================] - 51s 110


Epoch 00069: val_acc did not improve from 0.87980
Epoch 70/200
Learning rate:  0.001
468/468 [==============================] - 51s 108ms/step - loss: 0.3802 - acc: 0.9416 - val_loss: 0.6015 - val_acc: 0.8752

Epoch 00070: val_acc did not improve from 0.87980
Epoch 71/200
Learning rate:  0.001
468/468 [==============================] - 51s 108ms/step - loss: 0.3807 - acc: 0.9411 - val_loss: 0.8245 - val_acc: 0.8313

Epoch 00071: val_acc did not improve from 0.87980
Epoch 72/200
Learning rate:  0.001
468/468 [==============================] - 50s 108ms/step - loss: 0.3799 - acc: 0.9418 - val_loss: 0.7234 - val_acc: 0.8547

Epoch 00072: val_acc did not improve from 0.87980
Epoch 73/200
Learning rate:  0.001
468/468 [==============================] - 50s 108ms/step - loss: 0.3857 - acc: 0.9394 - val_loss: 0.6358 - val_acc: 0.8723

Epoch 00073: val_acc did not improve from 0.87980
Epoch 74/200
Learning rate:  0.001
468/468 [==============================] - 50s 108ms/step - loss: 0.3724 -

468/468 [==============================] - 51s 108ms/step - loss: 0.1744 - acc: 0.9940 - val_loss: 0.4484 - val_acc: 0.9286

Epoch 00104: val_acc did not improve from 0.92960
Epoch 105/200
Learning rate:  0.0001
468/468 [==============================] - 51s 108ms/step - loss: 0.1740 - acc: 0.9935 - val_loss: 0.4677 - val_acc: 0.9247

Epoch 00105: val_acc did not improve from 0.92960
Epoch 106/200
Learning rate:  0.0001
468/468 [==============================] - 51s 108ms/step - loss: 0.1711 - acc: 0.9941 - val_loss: 0.4615 - val_acc: 0.9280

Epoch 00106: val_acc did not improve from 0.92960
Epoch 107/200
Learning rate:  0.0001
468/468 [==============================] - 50s 108ms/step - loss: 0.1690 - acc: 0.9941 - val_loss: 0.4539 - val_acc: 0.9280

Epoch 00107: val_acc did not improve from 0.92960
Epoch 108/200
Learning rate:  0.0001
468/468 [==============================] - 50s 108ms/step - loss: 0.1665 - acc: 0.9941 - val_loss: 0.4597 - val_acc: 0.9259

Epoch 00108: val_acc did no

KeyboardInterrupt: 